# <center> ***Proyecto Integrador*** </center>
## <center> ***Avance 4*** </center>
### Profesores: Dra. Grettel Barceló, Dr. Luis Enrique Falcón.
### Tutor: Dr Carlos Alberto Villaseñor Padilla
### Alumnos:

* Aurelio Antonio Lozano Rábago      A01081266


#### <p style='text-align: right;'> 30 de Mayo del 2024 </p>

Importar bibliotecas



In [17]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet, InceptionV3, VGG16, ResNet50, DenseNet121,EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
import torch
import tensorflow as tf

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Importar MobileNet base pero descartaremos las últimas capa

In [19]:
# Lista de modelos
base_models = [
    MobileNet(weights='imagenet', include_top=False),
    InceptionV3(weights='imagenet', include_top=False),
    VGG16(weights='imagenet', include_top=False),
    ResNet50(weights='imagenet', include_top=False),
    DenseNet121(weights='imagenet', include_top=False),
    EfficientNetB0(weights='imagenet', include_top=False)
]


C:\Users\44942\AppData\Local\Temp\ipykernel_14812\3195174664.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  MobileNet(weights='imagenet', include_top=False),


Crear nueva Red Neuronal y pegar redes neuronales

In [20]:
def add_custom_layers(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    preds = Dense(3, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=preds)
    return model

Crear generador desde carpeta

In [21]:
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

train_generator = train_datagen.flow_from_directory('organized_data/train/',
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=50,
                                                    class_mode='categorical',
                                                    shuffle=True)

val_generator = val_datagen.flow_from_directory('organized_data/val/',
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=25,
                                                class_mode='categorical',
                                                shuffle=True)

Found 3002 images belonging to 3 classes.
Found 823 images belonging to 3 classes.


In [22]:
print(f'Número de imágenes de entrenamiento: {train_generator.samples}')
print(f'Número de imágenes de validación: {val_generator.samples}')

Número de imágenes de entrenamiento: 3002
Número de imágenes de validación: 823


Compilar Modelo y entrenar desde carpeta

In [23]:
histories = []
for i, base_model in enumerate(base_models, start=1):
    print ("Modelo: ",i)
    model = add_custom_layers(base_model)

    # Congelar capas base
    for layer in base_model.layers:
        layer.trainable = False
    for layer in model.layers[-5:]:
        layer.trainable = True

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    step_size_train = train_generator.n // train_generator.batch_size
    step_size_val = val_generator.n // val_generator.batch_size

    history = model.fit(train_generator, validation_data=val_generator,
                        steps_per_epoch=step_size_train, validation_steps=step_size_val,
                        epochs=20)
    
    histories.append(history)

Modelo:  1
Epoch 1/20


c:\Users\44942\.conda\envs\new_gpu_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


60/60 ━━━━━━━━━━━━━━━━━━━━ 31s 472ms/step - accuracy: 0.4846 - loss: 1.1487 - val_accuracy: 0.8025 - val_loss: 0.5899
Epoch 2/20
 1/60 ━━━━━━━━━━━━━━━━━━━━ 21s 367ms/step - accuracy: 0.6600 - loss: 0.7115

c:\Users\44942\.conda\envs\new_gpu_env\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6600 - loss: 0.7115 - val_accuracy: 0.8261 - val_loss: 0.5427
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 28s 461ms/step - accuracy: 0.7816 - loss: 0.5647 - val_accuracy: 0.8363 - val_loss: 0.4149
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8000 - loss: 0.5493 - val_accuracy: 1.0000 - val_loss: 0.2273
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 29s 472ms/step - accuracy: 0.8420 - loss: 0.4085 - val_accuracy: 0.8863 - val_loss: 0.3196
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8800 - loss: 0.2771 - val_accuracy: 0.9130 - val_loss: 0.2362
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 28s 462ms/step - accuracy: 0.8695 - loss: 0.3370 - val_accuracy: 0.9050 - val_loss: 0.2872
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8400 - loss: 0.2618 - val_accuracy: 0.8261 - val_loss: 0.4092
Epoch 9/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 29s 471ms/step - accuracy: 0.8760 - loss: 0.3374 - val_accuracy: 0.9075 - val_loss

# Conclusiones

### Tabla comparativa de los 6 modelos:

| Modelo | Accuracy | Loss | Val Accuracy | Val Loss |
|--------|----------|------|--------------|----------|
| MobileNet | 0.8800 | 0.3585  | 0.8696 | 0.2450 |
| InceptionV3 |  0.8800 | 0.3147 | 0.9565 | 0.2178 |
| VGG16 | 0.8600 | 0.2975 | 0.8696 | 0.3166 |
| ResNet50 | 0.8800 | 0.3894 | 0.9130 | 0.3361 |
| DenseNet121 |  0.8600 | 0.3072 | 0.9130 | 0.1998 |
| EfficientNetB0 | 0.3000 | 1.1023 | 0.3913 | 1.1144 |

### Tiempo de entrenamiento de cada modelo:

- **Modelo 1:** 104 minutos y 48 segundos
- **Modelo 2:** 91 minutos y 49 segundos
- **Modelo 3:** 65 minutos y 28 segundos
- **Modelo 4:** 89 minutos y 20 segundos
- **Modelo 5:** 103 minutos y 50 segundos
- **Modelo 6:** 13 minutos y 58 segundos

Mejor Modelo: Modelos 5 es el mejor, destacándose por su alta precisión y baja pérdida tanto en el entrenamiento como en la validación.